Connect to drive

In [ ]:
from google.colab import drive
drive.mount('/root/gdrive', force_remount=True)

Mounted at /root/gdrive


In [ ]:
%cd /root/gdrive/MyDrive/Bangla Newspaper Classification

/root/gdrive/MyDrive/Bangla Newspaper Classification


Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from collections import Counter

Load data

In [ ]:
import json

with open('data/data.json', encoding='utf-8') as fh:
    data = json.load(fh)
print(data[0])

{'author': 'গাজীপুর প্রতিনিধি', 'category': 'bangladesh', 'category_bn': 'বাংলাদেশ', 'published_date': '০৪ জুলাই ২০১৩, ২৩:২৬', 'modification_date': '০৪ জুলাই ২০১৩, ২৩:২৭', 'tag': ['গাজীপুর'], 'comment_count': 0, 'title': 'কালিয়াকৈরে টিফিন খেয়ে ৫০০ শ্রমিক অসুস্থ, বিক্ষোভ', 'url': 'http://www.prothom-alo.com/bangladesh/article/19030', 'content': 'গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায় আজ বৃহস্পতিবার রাতের টিফিন খেয়ে একটি পোশাক কারখানার ৫০০ শ্রমিক অসুস্থ হয়ে পড়েছেন। এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা।সফিপুর মডার্ন হাসপাতালের জরুরি বিভাগের চিকিত্সক আল আমিন প্রথম আলো ডটকমকে বলেন, খাদ্যে বিষক্রিয়ায় তাঁরা (শ্রমিকেরা) অসুস্থ হয়ে পড়েছেন। এতে আতঙ্কিত হওয়ার কিছু নেই। অসুস্থদের চিকিত্সা দেওয়া হয়েছে।কারখানার শ্রমিক ও পুলিশ সূত্রে জানা যায়, উপজেলার তেলিরচালা এলাকার সেজাদ সোয়েটার লিমিটেড কারখানার শ্রমিকদের আজ রাত সাড়ে সাতটার দিকে টিফিন দেওয়া হয়। টিফিনে ছিল ডিম, রুটি, পেটিস ও কলা। টিফিন খেয়ে শ্রমিকেরা যথারীতি কাজে যোগ দেন। ওই টিফিন খাওয়ার প্রায় এক ঘণ্টা পর রাত সা

In [ ]:
df = pd.read_json('data/data.json')
print(df.head()) 

NameError: ignored

Visualize data

In [ ]:
print("Total Rows:", len(data))
print("Total Cols:", len(data[0]))

Total Rows: 437948
Total Cols: 10


In [ ]:
print(type(data))
print("All columns:")
for col in data[0]:
  print(col)

<class 'list'>
All columns:
author
category
category_bn
published_date
modification_date
tag
comment_count
title
url
content


Retrieve all catagories

In [ ]:
all_categories = set([sample['category'] for sample in data])
print(all_categories)
print(len(all_categories))

{'bs-events', 'northamerica', 'durporobash', 'events', 'opinion', 'trust', 'diverse', 'education', 'kishoralo', '-1', 'international', 'pachmisheli', 'tarunno', 'life-style', 'bondhushava', '22221', 'chakri-bakri', 'nagorik-kantho', 'economy', 'bangladesh', 'entertainment', 'onnoalo', 'AskEditor', 'special-supplement', 'roshalo', 'we-are', 'facebook', 'mpaward1', 'demo-content', 'protichinta', 'sports', 'technology'}
32


test code

In [ ]:
a = []
print(a)
a.append({'c',7})
print(a)

[{}]
[{}, {'c', 7}]


Sorting based on category count

In [ ]:
category_count_list = []
count = 0
for cat in all_categories:
  count = 0
  for row in data:
    if row['category'] == cat:
      count += 1
  category_count_list.append({'category': cat, 'count': count})
print(category_count_list)

from operator import itemgetter
sorted_cat_count_list = []
# sorted_cat_count_list = sorted(category_count_list, key=lambda x: x.count, reverse=True)
category_count_list.sort(key = itemgetter('count'), reverse=True)
print(category_count_list)

[{'category': 'bs-events', 'count': 1}, {'category': 'northamerica', 'count': 6990}, {'category': 'durporobash', 'count': 7402}, {'category': 'events', 'count': 2}, {'category': 'opinion', 'count': 15699}, {'category': 'trust', 'count': 443}, {'category': 'diverse', 'count': 2}, {'category': 'education', 'count': 9721}, {'category': 'kishoralo', 'count': 508}, {'category': '-1', 'count': 123}, {'category': 'international', 'count': 30856}, {'category': 'pachmisheli', 'count': 3443}, {'category': 'tarunno', 'count': 40}, {'category': 'life-style', 'count': 10852}, {'category': 'bondhushava', 'count': 990}, {'category': '22221', 'count': 11}, {'category': 'chakri-bakri', 'count': 75}, {'category': 'nagorik-kantho', 'count': 83}, {'category': 'economy', 'count': 17245}, {'category': 'bangladesh', 'count': 232504}, {'category': 'entertainment', 'count': 30466}, {'category': 'onnoalo', 'count': 2702}, {'category': 'AskEditor', 'count': 1}, {'category': 'special-supplement', 'count': 859}, {

Discard low count catagories

In [ ]:
selected_cats = []

for p in category_count_list:
    if p['count'] > 1000:
        selected_cats.append(p['category'])
print(selected_cats)
print(len(selected_cats))

['bangladesh', 'sports', 'international', 'entertainment', 'economy', 'opinion', 'technology', 'life-style', 'education', 'durporobash', 'northamerica', 'pachmisheli', 'we-are', 'onnoalo', 'roshalo']
15


Prepare data X, Y

In [ ]:
x_text = []
y_label = []

for row in data:
    if row['category'] in selected_cats:
        y_label.append(row['category'])
        x_text.append(row['content'])
print(len(x_text),len(y_label))

434611 434611


One hot encoding of Y

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# Label Encode
encoder = LabelEncoder()
class_labels = encoder.fit_transform(y_label)
print(class_labels,len(class_labels),class_labels.shape)
print(set(class_labels))

#One hot Encode
encoder = OneHotEncoder(sparse=False)
class_labels = class_labels.reshape((class_labels.shape[0], 1))
y_ohe = encoder.fit_transform(class_labels)
print(y_ohe,y_ohe.shape)

[ 0 12  0 ... 12 12  0] 434611 (434611,)
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]] (434611, 15)


In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_text)
x_token = tokenizer.texts_to_sequences(x_text)
vocab_size = len(tokenizer.word_index) + 1
# print(x_token,type(x_token),len(x_token))

In [ ]:
print(type(x_token),len(x_token))

<class 'list'> 434611


In [ ]:
from keras.preprocessing.sequence import pad_sequences
maxlen = 300
x_pad = pad_sequences(x_token, padding='post', maxlen=maxlen)
print(type(x_pad),len(x_pad),x_pad.shape)

<class 'numpy.ndarray'> 434611 (434611, 300)


In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Bidirectional, Dense

embedding_dim = 8

model = Sequential()
model.add(Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(Bidirectional(LSTM(128, return_sequences = True)))
model.add(Bidirectional(LSTM(128))) 
model.add(Dense(y_ohe.shape[1], activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 300, 8)            21842200  
                                                                 
 bidirectional_8 (Bidirectio  (None, 300, 256)         140288    
 nal)                                                            
                                                                 
 bidirectional_9 (Bidirectio  (None, 256)              394240    
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 15)                3855      
                                                                 
Total params: 22,380,583
Trainable params: 22,380,583
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=2, test_size=0.3, random_state=0)
sss.get_n_splits(x_pad, y_ohe)

#print(sss)       

for train_index, test_index in sss.split(x_pad, y_ohe):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = x_pad[train_index], x_pad[test_index]
    y_train, y_test = y_ohe[train_index], y_ohe[test_index]

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    verbose=1,
                    validation_split=0.2,
                    batch_size=256
                    )

Epoch 1/10
951/951 [==============================] - 8414s 9s/step - loss: 0.9629 - accuracy: 0.7159 - val_loss: 0.7137 - val_accuracy: 0.7955
Epoch 2/10
951/951 [==============================] - 8115s 9s/step - loss: 0.4915 - accuracy: 0.8566 - val_loss: 0.4692 - val_accuracy: 0.8637
Epoch 3/10
951/951 [==============================] - 7955s 8s/step - loss: 0.3090 - accuracy: 0.9083 - val_loss: 0.4392 - val_accuracy: 0.8702
Epoch 4/10
951/951 [==============================] - ETA: 0s - loss: 0.1781 - accuracy: 0.9466

**LSTM train & valid**

Epoch 1/10
951/951 [==============================] - 8414s 9s/step - loss: 0.9629 - accuracy: 0.7159 - val_loss: 0.7137 - val_accuracy: 0.7955
Epoch 2/10
951/951 [==============================] - 8115s 9s/step - loss: 0.4915 - accuracy: 0.8566 - val_loss: 0.4692 - val_accuracy: 0.8637
Epoch 3/10
951/951 [==============================] - 7955s 8s/step - loss: 0.3090 - accuracy: 0.9083 - val_loss: 0.4392 - val_accuracy: 0.8702
Epoch 4/10
951/951 [==============================] - ETA: 0s - loss: 0.1781 - accuracy: 0.9466

In [ ]:
model.evaluate(X_test, y_test)

NameError: ignored

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.legend(['train', 'valid'])
plt.show()

NameError: ignored